In [1]:
from yahoo_fin import stock_info as si
from matplotlib import pyplot as plt 
import pandas_datareader as dr
import quandl
import pandas as pd
import numpy as np
from pandas_datareader import data as wb
import datetime
import yfinance as yf
from requests_html import HTMLSession


from sklearn.utils import shuffle
import numpy_financial as npf
'''----// General input variables //----'''


long_term_growth = 0.01 # динамическое значение инфляции для каждой страны
iterations = 1000


In [29]:
data_csv_ticker = pd.read_csv('TICKER_companies.csv')
data_csv = data_csv_ticker.set_index('country')
data_industry_sector = pd.read_excel('SECTORS.xlsx')

data_ticker = data_csv[['ticker', 'compnumber', 'mic']]
data_ticker['mic'].fillna('0.0')
data_ticker['mic'] = data_ticker['mic'].where(pd.notnull(data_ticker['mic']), '0.0', axis='rows') # NAN

CAN = data_ticker.loc['CAN']    # дергаем отсюда тикер и номер компании
DEU = data_ticker.loc['DEU']

#data_ticker['compnumber'] ## список компаний, будем брать отсюда


<ipython-input-29-415e544e7e7e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ticker['mic'] = data_ticker['mic'].where(pd.notnull(data_ticker['mic']), '0.0', axis='rows') # NAN


0    9.876500e+14
1    9.876500e+14
2    9.876500e+14
Name: amount, dtype: float64

In [3]:
listus =[]
for i in range(len(data_ticker.index)):
    listus.append(data_ticker['mic'][i][0:3])
data_ticker['mic_new'] = listus

<ipython-input-3-b47311ca4c77>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ticker['mic_new'] = listus


In [4]:
data_industry_sector
companys_sector = []
data_group_mic = data_ticker.set_index(['mic_new'])


[1.4]

In [31]:
sectors = data_industry_sector['Industry'].tolist()
sectors_mic = data_industry_sector['Industry_mic'].tolist()

overview_sectors = {}
symbols_all = {}


for sector, sectorz in zip(sectors[2:3], sectors_mic[2:3]):                          

    symbols = []
    symbols_all[sector] = {}
    overview_sectors[sector] = {}
    
    gross_Margin_sector = []
    operating_Margin_sector = []
    net_Profit_Margin_sector = [] 
    ROCE_sector = []
    ROA_sector = []
    ROE_DuPont_sector = []
    taxRate_sector = [] 
    
    
    

#         for sectorz in (data_industry_sector['Industry_mic'].tolist()):
    
    for companys in data_group_mic.loc[f'{sectorz}']['compnumber']:
        
        quandl.ApiConfig.api_key = 'c3baa8FMfeFWP_wpbhKL'
        data = pd.DataFrame(quandl.get_table('MER/F1', compnumber=companys, reporttype='A',  paginate=True))
        main_data = data.sort_values(by=['reportdate']).set_index('indicator')
        data_amount = main_data['amount']
        work_table = pd.DataFrame()
        
        # берем данные за 11 лет для расчета

        work_table['date'] = main_data['reportdate'].T.drop_duplicates().T[::-1].reset_index(drop=True)          
        

        year_len = len(work_table['date'].index)    
        if year_len > 10:
            year_len = 10
            
        
        try:
            balance = pd.DataFrame()
            balance['date'] = main_data['reportdate'].T.drop_duplicates().T[::-1].reset_index(drop=True)[:year_len+1]

            balance['totalAssets'], balance['stockholdersEquity'] \
                = data_amount['Total Assets'][::-1].reset_index(drop=True)[:year_len+1], data_amount['''Stockholders' Equity'''][::-1].reset_index(drop=True)[:year_len+1] 

        except:
            pass
        
        try:
            work_table['operatingMargin'] = data_amount['Operating Margin'][::-1].reset_index(drop=True)[:year_len] 
            work_table = work_table[:year_len].reset_index(drop=True).fillna(0)
            operating_Margin_10_year = work_table['operatingMargin'].mean()
#             print(f''' operating_Margin_10_year {work_table['operatingMargin']}''')

            if -50 < np.mean(operating_Margin_10_year) < 100:
#                 print(f''' operating_Margin_10_year {np.mean(operating_Margin_10_year)}''')
                operating_Margin_sector.append(operating_Margin_10_year/ 100)
            else:
                pass             
        except:
            pass
        
        try:
            work_table['gross_Margin'] = data_amount['Gross Margin'][::-1].reset_index(drop=True)[:year_len]
            work_table = work_table[:year_len].reset_index(drop=True).fillna(0)
            gross_Margin_10_year = work_table['gross_Margin'].mean()
                
            if -50 < np.mean(gross_Margin_10_year) < 100: 
                
                gross_Margin_sector.append(gross_Margin_10_year/ 100)
            else:
                pass             
        except:
            pass
        
        try:
            work_table['net_Margin'] = data_amount['Net Margin'][::-1].reset_index(drop=True)[:year_len]       
            work_table = work_table[:year_len].reset_index(drop=True).fillna(0)
            net_Margin_10_year = work_table['net_Margin'].mean() 
                
            if -50 < np.mean(net_Margin_10_year) < 100: 
                net_Profit_Margin_sector.append(net_Margin_10_year / 100)
            else:
                pass             
        except:
            pass
        
        
        # _________________ Считаем ROCE _________________

        try:
            work_table['longTermObligations'] = (
                        data_amount['Long Term Debt to Equity'][::-1].reset_index(drop=True)[:year_len] * data_amount[
                                                                                                        '''Stockholders' Equity'''][
                                                                                                    ::-1].reset_index(
                    drop=True)[:year_len])
            work_table['operatingIncome'] = data_amount['Operating Income'][::-1].reset_index(drop=True)[:year_len]
            work_table['stockholdersEquity'] = data_amount['''Stockholders' Equity'''][::-1].reset_index(drop=True)[:year_len]
            work_table = work_table[:year_len].reset_index(drop=True).fillna(0)
            ROCE_10_year = []

            for i in range(year_len - 1):
                ROCE = work_table['operatingIncome'][i] / (
                        (work_table['stockholdersEquity'][i + 1] + work_table['longTermObligations'][i + 1]) \
                        + (work_table['stockholdersEquity'][i] + work_table['longTermObligations'][i]) * 0.5)
                ROCE_10_year.append(ROCE)

            if -50 < np.mean(ROCE_10_year) < 100:
                #                 print(f''' operating_Margin_10_year {np.mean(operating_Margin_10_year)}''')
                ROCE_sector.append(np.mean(ROCE_10_year))
            else:
                pass

        except:
            pass
        # _________________ Считаем ROA _________________

        try:
            work_table['netIncome'] = data_amount['Net Income'][::-1].reset_index(drop=True)[:year_len]
            work_table['totalAssets'] = data_amount['Total Assets'][::-1].reset_index(drop=True)[:year_len]
            work_table = work_table[:year_len].reset_index(drop=True).fillna(0)

            ROA_10_year = np.mean(work_table['netIncome'] / work_table['totalAssets'])

            if -50 < np.mean(ROA_10_year) < 100:
                #                 print(f''' operating_Margin_10_year {np.mean(operating_Margin_10_year)}''')
                ROA_sector.append(ROA_10_year)
            else:
                pass

        except:
            pass

        # _________________ Считаем ROE_DuPont _________________

        try:
            work_table['netIncome'] = data_amount['Net Income'][::-1].reset_index(drop=True)[:year_len]
            work_table['totalAssets'] = data_amount['Total Assets'][::-1].reset_index(drop=True)[:year_len]
            work_table['revenue'] = data_amount['Total Revenue'][::-1].reset_index(drop=True)[:year_len]
            work_table['stockholdersEquity'] = data_amount['''Stockholders' Equity'''][::-1].reset_index(drop=True)[:year_len]

            work_table = work_table[:year_len].reset_index(drop=True).fillna(0)

            Profitability = []
            TechnicalEfficiency = []
            FinancialStructure = []

            year_range_Dup = 5
            if year_len < 5:
                year_range_Dup = year_len

            for year in range(0, year_range_Dup):
                last_year = year + 1
                Profitability.append(work_table['netIncome'][year] / work_table['revenue'][year])
                TechnicalEfficiency.append(work_table['revenue'][year] / (
                        (work_table['totalAssets'][year] + balance['totalAssets'].replace(np.nan, 0)[last_year]) / 2))
                FinancialStructure.append(
                    ((work_table['totalAssets'][year] + balance['totalAssets'].replace(np.nan, 0)[last_year]) / 2) /
                    ((work_table['stockholdersEquity'][year] +
                      balance['stockholdersEquity'].replace(np.nan, 0)[last_year]) / 2))

            table = pd.DataFrame()
            table['Profitability'] = Profitability
            table['TechnicalEfficiency'] = TechnicalEfficiency
            table['FinancialStructure'] = FinancialStructure

            work_table['ROE_DuPont'] = table['FinancialStructure'] * table['TechnicalEfficiency'] * \
                                       table['Profitability']

            ROE_DuPont = work_table['ROE_DuPont'].mean(axis=0)

            if -0.5 < np.mean(ROE_DuPont) < 1:
                #                 print(f''' operating_Margin_10_year {np.mean(operating_Margin_10_year)}''')
                ROE_DuPont_sector.append(ROE_DuPont)
            else:
                pass
        except:
            pass

        # _________________ Считаем taxRate _________________

        try:
            work_table['taxRate'] = data_amount['Calculated Tax Rate'][::-1].reset_index(drop=True)[:year_len]

            work_table = work_table[:year_len].reset_index(drop=True).fillna(0)

            taxRate_10_year = work_table['taxRate'].mean()

            if -50 < np.mean(taxRate_10_year) < 100:
                #                 print(f''' operating_Margin_10_year {np.mean(operating_Margin_10_year)}''')
                taxRate_sector.append(taxRate_10_year / 100)
            else:
                pass

        except:
            pass
        
        
#             print('*'*50)   

#             print(operating_Margin_sector)

            
    overview_sectors[sector]['Operating Margin'] = np.mean(operating_Margin_sector)
    overview_sectors[sector]['Gross Margin'] = np.mean(gross_Margin_sector)
    overview_sectors[sector]['Net Profit Margin'] = np.mean(net_Profit_Margin_sector)
    overview_sectors[sector]['ROCE'] = np.mean(ROCE_sector)
    overview_sectors[sector]['ROA'] = np.mean(ROA_sector)
    overview_sectors[sector]['ROE DuPont'] = np.mean(ROE_DuPont_sector)
    overview_sectors[sector]['Tax Rate'] = np.mean(taxRate_sector)
        
overview_sectors 
        
# np.mean(operatingMargin_sector)     

# data_ticker['compnumber'][0:10]

{'Tobacco': {'Operating Margin': 0.2156403088253968,
  'Gross Margin': 0.4177341766035354,
  'Net Profit Margin': 0.14921841000529099,
  'ROCE': 0.20857827428873463,
  'ROA': 0.01022852554028693,
  'ROE DuPont': 0.19103743097404827,
  'Tax Rate': 0.272839731271605}}

In [33]:
sector_margin_company = overview_sectors
sector_margin_company = pd.DataFrame(sector_margin_company)
pd.to_pickle(sector_margin_company, "sector_margin_qandl.pkl")

In [34]:
sector_margin_company = pd.read_pickle("sector_margin_qandl.pkl")
sector_margin_company = pd.DataFrame(sector_margin_company)
sector_margin_company

,Tobacco
Gross Margin,0.417734
Net Profit Margin,0.149218
Operating Margin,0.215640
ROA,0.010229
ROCE,0.208578
ROE DuPont,0.191037
Tax Rate,0.272840


In [8]:

industry_dict =dict(zip(data_industry_sector['Industry'].tolist(), data_industry_sector['Industry_mic'].tolist()))
industry_pre_df = pd.DataFrame(industry_dict, index=[0])
industry_pre_df

,Agriculture,Food & Beverage,Tobacco,Textiles & Apparel,Appliances & Electronics,Consumer Goods,Household & Personal Products,Autos & Auto Parts,Retail,Consumer Services,...,Electrical Equipment,Transport,Commercial Services & Supplies,Paper & Forest Products,Metals & Mining,Chemicals,Synthetic Materials,Construction Materials,Oil & Gas,Miscellaneous Industry
0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,2.1,2.2,...,7.3,7.4,7.5,8.1,8.2,8.3,8.4,8.5,9.1,0.0
